In [ ]:
import logging

handler = logging.StreamHandler() 
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
logging.getLogger().addHandler(handler)

logging.getLogger("helpers").setLevel("DEBUG")
logger = logging.getLogger(__name__)
logger.setLevel("DEBUG")

In [ ]:
import pandas as pd

In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds

In [ ]:
from google.cloud import bigquery

In [ ]:
client = bigquery.Client()

In [ ]:
query_string = """
SELECT
    aliquot_barcode,
    gene_name,
    sum(HTSeq__FPKM) as HTSeq__FPKM_sum
FROM `isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current`
WHERE
    project_short_name = "TCGA-SKCM"
    and gene_type = 'protein_coding'
GROUP BY 1, 2
"""

In [ ]:
query_job = client.query(query_string)

In [ ]:
df = query_job.to_dataframe(progress_bar_type="tqdm")
# .to_arrow(progress_bar_type="tqdm")\
# .to_pandas(
#     strings_to_categorical=True,
#     self_destruct=True
# )

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df_1 = df.pivot(index="gene_name", columns="aliquot_barcode", values="HTSeq__FPKM_sum")

In [ ]:
df_1.head()

In [ ]:
df[df["gene_name"] == "SMIM11"].set_index(["aliquot_barcode", "gene_name", "Ensembl_gene_id"])[["HTSeq__FPKM"]].sort_index().head(30)

In [ ]:
df_piv = df.(
    index=["gene_name", "Ensembl_gene_id_v"],
    columns="aliquot_barcode",
    values="HTSeq__FPKM",
)

In [ ]:
df_piv["TCGA-3N-A9WB-06A-11R-A38C-07"].

In [ ]:
(df[df["gene_name"] == "SMIM11"]).groupby("Ensembl_gene_id_v")["HTSeq__FPKM"]\
.aggregate(count=len, zero_frac=lambda x: (x == 0).mean()).sort_values(by="count")

In [ ]:
_ = df.groupby(["gene_name"])["Ensembl_gene_id_v"].agg("count")
_[_ > 472].sort_values()

In [ ]:
# df[df["gene"name" ]\
# .groupby(["aliquot_barcode", "gene_name"])\
# ["HTSeq__FPKM"]\
# .agg("count")\
# .sort_values(ascending=False)\
# .head(50)

In [ ]:
df.head(30000).set_index(["aliquot_barcode", "gene_name"])

In [ ]:
df_2 = query_job.to_dataframe(progress_bar_type="tqdm")

In [ ]:
df_2.info()

In [ ]:
df.info()

In [1]:
import importlib

import helpers

In [2]:
importlib.reload(helpers)

<module 'helpers' from '/home/jupyter/deconv/helpers/__init__.py'>

In [3]:
df_3 = helpers.datasets.load_tcga_skcm_bigquery()

Downloading: 100%|██████████| 9304064/9304064 [00:04<00:00, 2300628.10rows/s]


In [5]:
df_3.shape

(19712, 472)